## 라이브러리 임포트

In [3]:
import random
import numpy as np
import copy
import matplotlib.pyplot as plt
import pandas
import scipy.ndimage as ndimage

## 0430

### 스핀 행렬

In [4]:
def mat(L):
    data = np.random.choice([-1, 1], size=(L, L)) # -1과 1 중에서 무작위로 LxL 개 뽑음
    return data

In [5]:
# 코드 작성에 활용할 연습용 데이터 a
a = mat(5)
print(a)

[[ 1 -1  1  1  1]
 [ 1  1  1  1  1]
 [-1 -1 -1 -1  1]
 [ 1 -1  1 -1  1]
 [-1  1  1  1 -1]]


### (n번째) 노드의 이웃 노드

In [6]:
def neighbor(array, n): # n번째 노드의 이웃 노드 인덱스(상하좌우 4개)를 찾음
    L, _ = array.shape

    # 상
    up = n-L # 윗쪽으로 한 칸 이동
    if up < 0: # 이동 전이 맨 윗쪽 열인 경우,
        up += L*L # 아랫쪽으로 L칸 이동하여 주기적 경계 조건 만족

    # 하
    down = n+L # 아랫쪽으로 한 칸 이동
    if down >= L*L: # 이동 전이 맨 아랫쪽 열인 경우,
        down -= L*L # 윗쪽으로 L칸 이동하여 주기적 경계 조건 만족

    # 좌
    left = n-1 # 왼쪽으로 한 칸 이동
    if (left+1)%L == 0: # 이동 전이 맨 왼쪽 열인 경우,
        left += L # 오른쪽으로 L칸 이동(= 아래로 한 칸 이동)하여 주기적 경계 조건 만족
    # 우
    right = n+1 # 오른쪽으로 한 칸 이동
    if right%L == 0: # 이동 후가 맨 왼쪽 열(= 이동 전이 맨 오른쪽 열)인 경우,
        right -= L # 왼쪽으로 L칸 이동(= 위로 한 칸 이동)하여 주기적 경계 조건 만족

    return [up, down, left, right] # 이웃 노드의 인덱스 리스트(상하좌우 순) 출력

In [7]:
neighbor(a, 9) # 5x5 행렬 9번 노드의 이웃: [4, 14, 8, 5]

[4, 14, 8, 5]

### 각 노드의 에너지

In [8]:
def nb_energy(array, n):
    array_list = [item for sublist in array for item in sublist] # 행렬을 하나의 리스트로
    e = 0 # 에너지 초기화

    neighbors = neighbor(array, n) # neighbors: n번째 노드의 이웃 노드 리스트

    for i in neighbors:
        if array_list[n] == array_list[i]: # n번째 노드와 이웃 노드가 같은 스핀일 경우
            e -= 1 # -1을 sum
        else: # 다른 스핀일 경우
            e += 1 # +1을 sum

    return e # n번째 노드가 갖는 총 에너지 반환

In [9]:
nb_energy(a, 9) # 5x5 행렬 9번 노드의 에너지

-4

### 행렬의 총 에너지

In [12]:
def E(array):
    N = len(array)**2 # N = L x L
    result = []

    for i in range(0,N): # 0~(N-1)번째 노드를 반복하여
        result.append(nb_energy(array, i)) # 각 노드의 총 에너지를 저장

    return sum(result)/2 # 각 노드의 총 에너지를 모두 더한 후, (a,b)와 (b,a)의 중복 제거를 위해 2로 나누어 반환

In [13]:
E(a)

-2.0

## 0502

### Metropolis

In [14]:
def func3(array, count, beta=1):
    mu = array
    E_list = []
    m_list = []

    for _ in range(count):
        E_mu = E(mu)


        nu_list = [item for sublist in copy.copy(mu) for item in sublist] # 행렬을 하나의 리스트로
        r = random.randint(0, len(nu_list)-1) # 무작위로 인덱스를 뽑음 0~N
        if nu_list[r] == -1: nu_list[r] = 1 # 해당 인덱스 스핀을 flip
        else: nu_list[r] = -1
        nu = np.array(nu_list).reshape((len(array), len(array))) # flip 된 스핀을 포함한 새로운 nu
        E_nu = E(nu)

        if E_mu >= E_nu: # mu의 에너지가 nu의 에너지보다 크다면 1의 확률로 nu를 accept
            mu = nu
            # print(f'>accept< index = {r}, E = {E(mu)}, m = {np.sum(mu)/len(mu)**2}')
            # 그때마다의 flip된 스핀의 인덱스, mu의 에너지, 평균 스핀값을 출력

        elif np.random.uniform(0,1) <= np.exp(beta*(E_mu-E_nu)): # 아닌 경우, exp(beta*(E_mu-E_nu))의 확률로 nu를 accept
            mu = nu
            # print(f'>accept< index = {r}, E = {E(mu)}, m = {np.sum(mu)/len(mu)**2}')

        elif np.random.uniform(0,1) > np.exp(beta*(E_mu-E_nu)): # 둘 다 아닌 경우, reject되고 mu는 그대로
            mu = mu
            # print(f'>reject< index = {r}, E = {E(mu)}, m = {np.sum(mu)/len(mu)**2}')

        E_list.append(E(mu))
        m_list.append(np.sum(mu)/len(mu)**2)

    return array, mu, E_list, m_list

In [15]:
'''
# beta = 1
_, _, E1, m1 = func3(mat(5), 1000)
plt.plot(E1)
plt.plot(m1) # 1로 수렴

# beta = 0
_, _, E2, m2 = func3(mat(5), 1000, 0)
plt.plot(E2)
plt.plot(m2) # 0에서 왔다갔다

# beta = 10
_, _, E3, m3 = func3(mat(5), 1000, 10)
plt.plot(E3)
plt.plot(m3) # 1로 수렴
'''

'\n# beta = 1\n_, _, E1, m1 = func3(mat(5), 1000)\nplt.plot(E1)\nplt.plot(m1) # 1로 수렴\n\n# beta = 0\n_, _, E2, m2 = func3(mat(5), 1000, 0)\nplt.plot(E2)\nplt.plot(m2) # 0에서 왔다갔다\n\n# beta = 10\n_, _, E3, m3 = func3(mat(5), 1000, 10)\nplt.plot(E3)\nplt.plot(m3) # 1로 수렴\n'

## 0514

### *가로축을 beta로 표현할 것. delta가 아니라 스핀의 평균 값을 그릴 것.*

### 이웃 노드들의 스핀 합

In [16]:
def nb_sig(array, n):
    array_list = [item for sublist in array for item in sublist] # 행렬을 하나의 리스트로
    e = 0 # 에너지 초기화

    neighbors = neighbor(array, n) # neighbors: n번째 노드의 이웃 노드 리스트

    for i in neighbors:
        e += array_list[i] # 이웃 노드들의 스핀을 sum

    return e # n번째 노드가 갖는 총 에너지 반환

In [17]:
nb_sig(a, 9)

4

### (개선된) Metropolis

In [18]:
def func(array, count, beta=1):
    mu = array # mu 초기화
    exps = [np.exp(-4*beta), np.exp(-8*beta)] # exp 계산을 한 번만
    E_list = []
    delta_list = []

    for _ in range(count):
        mu_list = [item for sublist in copy.copy(mu) for item in sublist] # 행렬을 하나의 리스트로
        k = random.randint(0, len(mu_list)-1) # 무작위로 인덱스를 뽑음 0~N

        delta = 2 * mu_list[k] * nb_sig(mu,k) # delta E = 2 * 해당 노드 스핀 * 이웃 노드 스핀 합

        if delta <= 0:
            mu_list[k] = -mu_list[k]
        elif delta == 4 and np.random.uniform(0,1) <= exps[0]:
            mu_list[k] = -mu_list[k]
        elif delta == 8 and np.random.uniform(0,1) <= exps[1]:
            mu_list[k] = -mu_list[k]

        mu = np.array(mu_list).reshape((len(array), len(array))) # flip 된 스핀을 포함한 새로운 mu

        E_list.append(E(mu))
        delta_list.append(delta)

    return E_list, delta_list

In [19]:
'''
# beta = 1
E1, delta1 = func(mat(5), 1000)
print(E1, '\n', delta1)
plt.plot(E1)
plt.plot(delta1)

# beta = 0
E0, delta0 = func(mat(5), 1000, 0)
print(E0, '\n', delta0)

plt.plot(E0)
plt.plot(delta0)

avg_Es = []
avg_deltas = []

for i in range(0,11):
    e, d = func(mat(5), 10000, beta=0.1*i)
    avg_Es.append(np.mean(e))
    avg_deltas.append(abs(np.mean(d)))

print(avg_Es, '\n', avg_deltas)

plt.plot(avg_Es)
plt.plot(avg_deltas)
'''

"\n# beta = 1\nE1, delta1 = func(mat(5), 1000)\nprint(E1, '\n', delta1)\nplt.plot(E1)\nplt.plot(delta1)\n\n# beta = 0\nE0, delta0 = func(mat(5), 1000, 0)\nprint(E0, '\n', delta0)\n\nplt.plot(E0)\nplt.plot(delta0)\n\navg_Es = []\navg_deltas = []\n\nfor i in range(0,11):\n    e, d = func(mat(5), 10000, beta=0.1*i)\n    avg_Es.append(np.mean(e))\n    avg_deltas.append(abs(np.mean(d)))\n\nprint(avg_Es, '\n', avg_deltas)\n\nplt.plot(avg_Es)\nplt.plot(avg_deltas)\n"

## 0516

### Traveling salesman

In [ ]:
ncity = 100

# n개의 도시를 위한 2D 랜덤 좌표 생성
R = np.random.random((ncity, 2))
city = list(range(ncity))

def Distance(R1, R2):
    return np.linalg.norm(R1 - R2)

def TotalDistance(city, R):
    dist = 0
    for i in range(len(city) - 1):
        dist += Distance(R[city[i]], R[city[i + 1]])
    dist += Distance(R[city[-1]], R[city[0]])  # 시작 도시로 돌아오는 거리
    return dist

def Plot(city, R, dist):
    Pt = [R[city[i]] for i in range(len(city))]
    Pt += [R[city[0]]]
    Pt = np.array(Pt)
    plt.title('Total distance=' + str(dist))
    plt.plot(Pt[:, 0], Pt[:, 1], 'o-')
    plt.show()

def Swap(city, n):
    city_copy = city[:]
    current = city_copy[n - 1]
    # 교환
    city_copy[n - 1] = city_copy[n]
    city_copy[n] = current
    return city_copy

def CostSwap(R, city, n):
    current_cost = TotalDistance(city, R)
    swapped_city = Swap(city, n)
    new_cost = TotalDistance(swapped_city, R)
    return new_cost - current_cost, swapped_city

def TravelingSalesman(city, R, maxSteps, maxAccepted, Tstart, fCool, maxTsteps):
    T = Tstart
    dist = TotalDistance(city, R)
    for t in range(maxTsteps):
        accepted = 0
        for i in range(maxSteps):
            # 교환 시도
            n = np.random.randint(1, len(city))  # n이 유효한 범위 내에 있는지 확인
            de, new_city = CostSwap(R, city, n)
            if de < 0 or np.exp(-de / T) > np.random.rand():
                accepted += 1
                dist += de
                city = new_city  # 교환이 받아들여졌을 때만 도시 리스트 업데이트
            if accepted > maxAccepted:
                break
        T *= fCool
        Plot(city, R, dist)
        print("T=%10.5f , distance=%10.5f acc.steps=%d" % (T, dist, accepted))
        if accepted == 0:
            break
    return city

In [ ]:
'''
# 파라미터 설정
ncity = 10
maxSteps = 100 * ncity
maxAccepted = 10 * ncity
Tstart = 0.2
fCool = 0.9
maxTsteps = 100

np.random.seed(0)

R = np.random.random((ncity, 2))
city = list(range(ncity))

city = TravelingSalesman(city, R, maxSteps, maxAccepted, Tstart, fCool, maxTsteps)
'''

## 0521

### Ising spin

In [21]:
betas = np.arange(0.0, 1.1, 0.1).tolist() # x축 plot을 위해 beta 리스트 생성

In [24]:
def find_t(beta): # beta에 따른 tau를 구하는 함수
    E, _ = func(mat(5), 10000, beta)
    E = pandas.Series(E[200:]) # 초반 200개 번인
    ans = 0

    for i in range(1,501):
        ans += E.autocorr(lag=i) # lag을 1~500로 바꾸며 sum
    return ans

In [25]:
'''
taus = [] # beta에 따른 tau를 저장할 리스트
for beta in range(0, 11):
    ans = find_t(0.1*beta) # beta를 0.0~1.0로 바꾸며 tau를 구함
    taus.append(ans)
    print(f'Beta = {0.1 * beta}, Tau = {ans}')
'''

"\ntaus = [] # beta에 따른 tau를 저장할 리스트\nfor beta in range(0, 11):\n    ans = find_t(0.1*beta) # beta를 0.0~1.0로 바꾸며 tau를 구함\n    taus.append(ans)\n    print(f'Beta = {0.1 * beta}, Tau = {ans}')\n"

In [47]:
def find_error(L=5, beta_list=Betas, tau_list=taus):
    avg_Es = []
    avg_Ss = []
    err_Es = []
    err_Ss = []

    for b in range(0, len(beta_list)):
        e, s = func(mat(L), 10000, beta_list[b]) # beta를 바꾸며
        e = e[200:] # 초반 번인
        s = s[200:]
        avg_Es.append(np.mean(e)) # E의 평균
        avg_Ss.append(abs(np.mean(s))) # spin의 평균

        t = round(tau_list[b]) *2 # 각 L, beta에 따른 tau, 안정성을 위해 반올림 후 2를 곱해 주었음

        new_Es = e[::t] # step이 tau인 새로운 리스트
        new_Ss = s[::t]

        errE = np.std(new_Es)/np.sqrt(len(new_Es)-1) # 표준오차
        err_Es.append(errE)
        errS = np.std(new_Ss)/np.sqrt(len(new_Ss)-1)
        err_Ss.append(errS)

    return avg_Es, avg_Ss, err_Es, err_Ss

## 0523

### *critical region의 값들이 너무 불안정.*

### L=5일 때, L=10일 때에 magnetization의 그래프(오차 막대 포함)

In [48]:
Betas = np.arange(0.4, 0.5, 0.005).tolist() # x축 plot을 위해 beta 리스트 생성
Betas.insert(0, 0.3)
Betas.append(0.6)
Betas.insert(0, 0.0)
Betas.append(1.0)

In [49]:
def find_error(L=5, beta_list=Betas, tau_list=taus):
    avg_Es = []
    avg_Ss = []
    err_Es = []
    err_Ss = []

    for b in range(0, len(beta_list)):
        e, s = func(mat(L), 10000, beta_list[b]) # beta를 바꾸며
        e = e[200:] # 초반 번인
        s = s[200:]
        avg_Es.append(np.mean(e)) # E의 평균
        avg_Ss.append(abs(np.mean(s))) # spin의 평균

        t = round(tau_list[b]) *2 # 각 L, beta에 따른 tau, 안정성을 위해 반올림 후 2를 곱해 주었음

        new_Es = e[::t] # step이 tau인 새로운 리스트
        new_Ss = s[::t]

        errE = np.std(new_Es)/np.sqrt(len(new_Es)-1) # 표준오차
        err_Es.append(errE)
        errS = np.std(new_Ss)/np.sqrt(len(new_Ss)-1)
        err_Ss.append(errS)

    return avg_Es, avg_Ss, err_Es, err_Ss

In [50]:
'''
avg_Es, avg_Ss, err_Es, err_Ss = find_error(L=5, beta_list=Betas, tau_list=taus)

# E의 평균
plt.xlabel('Beta')
plt.ylabel('E')
plt.errorbar(betas, avg_Es, err_Es)

# spin의 평균
plt.xlabel('Beta')
plt.ylabel('S')
plt.errorbar(betas, avg_Ss, err_Ss)
'''

"\navg_Es, avg_Ss, err_Es, err_Ss = find_error(L=5, beta_list=Betas, tau_list=taus)\n\n# E의 평균\nplt.xlabel('Beta')\nplt.ylabel('E')\nplt.errorbar(betas, avg_Es, err_Es)\n\n# spin의 평균\nplt.xlabel('Beta')\nplt.ylabel('S')\nplt.errorbar(betas, avg_Ss, err_Ss)\n"

## 0528

### BFS

In [51]:
import queue

In [52]:
def bfs(K, nn, start, discovered, distances):
    q = queue.Queue()
    q.put(start) # 빈 큐에 출발점 추가하고
    discovered[start] = 1 # 출발점을 발견된 노드에 추가
    distances[start] = 0 # 출발점과의 거리는 0
    while not q.empty(): # 큐에 노드가 남아있는 동안
        element = q.get() # 큐의 맨 앞 노드를 원소로 하여
        for k in range(K[element]):
            neighbor = nn[element][k] # 그 이웃 각각에 대해
            if not discovered[neighbor]: # 미발견 노드인 경우
                q.put(neighbor) # 이웃 노드를 큐에 추가하고
                discovered[neighbor] = 1 # 이웃 노드를 발견된 노드에 추가
                distances[neighbor] = distances[element] + 1 # 원소의 이웃이므로 출발점까지의 거리는 (원소의 거리 +1)

In [53]:
# 파라미터 설정
N, Kmax = 8, 4
K = np.array([1, 2, 4, 1, 1, 3, 1, 1]) # 각 노드의 이웃의 수
nn = np.empty((N, Kmax), int) # 이웃 배열

# nn 배열을 샘플 데이터로 채우기
nn[0] = [2, -1, -1, -1]
nn[1] = [2, 5, -1, -1]
nn[2] = [0, 1, 3, 4]
nn[3] = [2, -1, -1, -1]
nn[4] = [2, -1, -1, -1]
nn[5] = [1, 6, 7, -1]
nn[6] = [5, -1, -1, -1]
nn[7] = [5, -1, -1, -1]

In [54]:
# discovered 배열 초기화
discovered = np.zeros(N, dtype=int)
# distances 배열 초기화
distances = np.full(N, -1, dtype=int)

# BFS 수행
bfs(K, nn, 0, discovered, distances)

# discovered 배열 출력
print("발견된 노드들:", discovered)
# distances 배열 출력
print("노드별 거리:", distances)

발견된 노드들: [1 1 1 1 1 1 1 1]
노드별 거리: [0 2 1 2 2 3 4 4]


### DFS

In [55]:
def dfs(K, nn, node, discovered):
    discovered[node] = 1
    for k in range(K[node]):
        neighbor = nn[node][k]
        if not discovered[neighbor]:
            dfs(K, nn, neighbor, discovered)

In [56]:
L = 5

N , Kmax = L**2, 4
K = np.empty(N, int)
for i in range(N):
    K[i] = Kmax
nn = np.empty((N, Kmax), int) # 이웃 배열
for i in range(N):
    nn[i][0] = i-1+L*(i%L==0) # left
    nn[i][1] = i+1-L*(i%L==L-1) # right
    nn[i][2] = i-L+N*(i//L==0) # up
    nn[i][3] = i+L-N*(i//L==L-1) # up

In [57]:
# discovered 배열 초기화
discovered = np.zeros(N, dtype=int)

# BFS 수행
dfs(K, nn, 0, discovered)

# discovered 배열 출력
print("발견된 노드들:", discovered)

발견된 노드들: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


## 0530

### (x,y) 노드의 이웃 노드

In [58]:
# 주기적 경계 조건을 처리하기 위한 이웃 함수 정의
def get_neighbors(i, j, L):
    return [
        ((i-1) % L, j),  # 위쪽
        ((i+1) % L, j),  # 아래쪽
        (i, (j-1) % L),  # 왼쪽
        (i, (j+1) % L)   # 오른쪽
    ]

### Wolff 알고리즘

In [59]:
def wolff_step(array, count, beta):
    L = len(array)
    p_add = 1 - np.exp(-2 * beta)  # P = 1 - exp(-2B)
    S_list = []  # <S>

    for _ in range(count):
        i, j = random.randint(0, L-1), random.randint(0, L-1) # 랜덤 스핀 좌표
        cluster_spin = array[i, j]
        stack = [(i, j)] # 스택과 클러스터에 각각 추가
        cluster = set([(i, j)])

        while stack:
            x, y = stack.pop() # LIFO
            for nx, ny in get_neighbors(x, y, L): # 해당 스핀의 이웃들 중
                if (nx, ny) not in cluster and array[nx, ny] == cluster_spin: # 아직 읽지 않은 스핀이고 해당 스핀과 같은 스핀이라면
                    if random.random() < p_add: # P의 확률로 스택과 클러스터에 추가
                        stack.append((nx, ny))
                        cluster.add((nx, ny))

        # Flip
        for x, y in cluster:
            array[x, y] = -array[x, y]

        # <S> 계산
        magnetization = np.sum(array) / (L * L)
        S_list.append(magnetization)

    return S_list

In [60]:
'''
S_list01 = wolff_step(spins, 1000, 0.1)

# Plot the results
plt.plot(S_list01, label='beta = 0.1')
plt.axhline(y=np.mean(S_list01), color='r', linestyle='--')
plt.xlabel('Steps')
plt.ylabel('<S>')
plt.legend()
plt.show()

Betas = np.arange(0.0, 1.1, 0.1).tolist()
S_list1 = []

for beta in Betas:
    s = wolff_step(spins, 10000, beta)
    S_list1.append(np.mean(np.abs(s)))
print(S_list1)

plt.scatter(Betas, S_list1, marker='x')
plt.xlabel('beta')
plt.ylabel('avg |<S>|')
'''

"\nS_list01 = wolff_step(spins, 1000, 0.1)\n\n# Plot the results\nplt.plot(S_list01, label='beta = 0.1')\nplt.axhline(y=np.mean(S_list01), color='r', linestyle='--')\nplt.xlabel('Steps')\nplt.ylabel('<S>')\nplt.legend()\nplt.show()\n\nBetas = np.arange(0.0, 1.1, 0.1).tolist()\nS_list1 = []\n\nfor beta in Betas:\n    s = wolff_step(spins, 10000, beta)\n    S_list1.append(np.mean(np.abs(s)))\nprint(S_list1)\n\nplt.scatter(Betas, S_list1, marker='x')\nplt.xlabel('beta')\nplt.ylabel('avg |<S>|')\n"

## 0604

### Union-find

In [61]:
# LxL 행렬을 -1과 1로 무작위하게 채우는 함수
def mat(L):
    data = np.random.choice([-1, 1], size=(L, L))
    return data

# 루트를 찾는 함수
def find(i):
    if ptr[i] < 0:
        return i
    else:
        ptr[i] = find(ptr[i]) # 경로 압축
        return ptr[i]

# 두 집합을 합치는 함수
def union(r1, r2, big):
    if r2 != r1:
        if ptr[r1] > ptr[r2]: # 작은 트리를 큰 트리 아래에 붙임
            ptr[r2] += ptr[r1]
            ptr[r1] = r2
            r1 = r2
        else:
            ptr[r1] += ptr[r2]
            ptr[r2] = r1

        if -ptr[r1] > big: # 기존의 big보다 크면 갱신
            big = -ptr[r1]
    return r1, r2, bi

In [62]:
'''
# 행렬 5 x 5
L = 5
test = mat(L)
N = L*L  # 요소의 수
Kmax = 4  # 최대 이웃 수

# 행렬을 하나의 리스트로 변환
ptr = [item for sublist in test for item in sublist]

# 이웃 배열
nn = np.empty((N, Kmax), int)
for i in range(N):
    nn[i][0] = i-1+L*(i%L==0) # left
    nn[i][1] = i+1-L*(i%L==L-1) # right
    nn[i][2] = i-L+N*(i//L==0) # up
    nn[i][3] = i+L-N*(i//L==L-1) # up

# 점을 채워나갈 순서 결정
order = np.arange(N)
np.random.shuffle(order)

#그래프 데이터를 저장할 리스트
x_data = []
y_data = []

big = 0
ptr = [-N-1] * N  # 처음엔 모두 빈 칸

# Union-Find 알고리즘 실행
for i in range(N):
    r1, s1 = order[i], order[i] # 위의 order를 따라 점 추가
    ptr[s1] = -1 # 그 크기는 1
    for j in range(Kmax):
        s2 = nn[s1, j] # 이웃 점들에 대해
        if ptr[s2] != -N-1: # 만일 비어있지 않으면
            r2 = find(s2) # 작은 쪽을 합친다
            r1, r2, big = union(r1, r2, big)

    # 탐색한 노드의 비율과 클러스터 사이즈의 비율 계산
    x_data.append((i+1) / N)
    y_data.append(big / N)

print("최종 big 값:", big)

# 그래프 그리기
plt.plot(x_data, y_data)
plt.xlabel('Node counts ratio')
plt.ylabel('Cluster size ratio')
plt.title(f'L = {L}')
plt.grid(True)
plt.show()
'''

'\n# 행렬 5 x 5\nL = 5\ntest = mat(L)\nN = L*L  # 요소의 수\nKmax = 4  # 최대 이웃 수\n\n# 행렬을 하나의 리스트로 변환\nptr = [item for sublist in test for item in sublist]\n\n# 이웃 배열\nnn = np.empty((N, Kmax), int)\nfor i in range(N):\n    nn[i][0] = i-1+L*(i%L==0) # left\n    nn[i][1] = i+1-L*(i%L==L-1) # right\n    nn[i][2] = i-L+N*(i//L==0) # up\n    nn[i][3] = i+L-N*(i//L==L-1) # up\n\n# 점을 채워나갈 순서 결정\norder = np.arange(N)\nnp.random.shuffle(order)\n\n#그래프 데이터를 저장할 리스트\nx_data = []\ny_data = []\n\nbig = 0\nptr = [-N-1] * N  # 처음엔 모두 빈 칸\n\n# Union-Find 알고리즘 실행\nfor i in range(N):\n    r1, s1 = order[i], order[i] # 위의 order를 따라 점 추가\n    ptr[s1] = -1 # 그 크기는 1\n    for j in range(Kmax):\n        s2 = nn[s1, j] # 이웃 점들에 대해\n        if ptr[s2] != -N-1: # 만일 비어있지 않으면\n            r2 = find(s2) # 작은 쪽을 합친다\n            r1, r2, big = union(r1, r2, big)\n\n    # 탐색한 노드의 비율과 클러스터 사이즈의 비율 계산\n    x_data.append((i+1) / N)\n    y_data.append(big / N)\n\nprint("최종 big 값:", big)\n\n# 그래프 그리기\nplt.plot(x_data, y_data)

## 0611

### Bond percolation

In [63]:
# 루트를 찾는 함수
def find(ptr, x):
    if ptr[x] < 0:
        return x
    else:
        ptr[x] = find(ptr, ptr[x]) # 경로 압축
        return ptr[x]

# 두 집합을 합치는 함수
def union(ptr, size, x, y):
    rootX = find(ptr, x)
    rootY = find(ptr, y)

    if rootX != rootY:
        if ptr[rootX] < ptr[rootY]:  # x의 규모가 더 클 때
            ptr[rootX] += ptr[rootY]
            ptr[rootY] = rootX
            size[rootX] += size[rootY]
        else:
            ptr[rootY] += ptr[rootX]
            ptr[rootX] = rootY
            size[rootY] += size[rootX]

# 4N/2 = 2N 개의 bond이므로 right, down 두 가지 방향으로 찾아본다
def bonds(L):
    bond_list = []
    for i in range(L):
        for j in range(L):
            node = i * L + j

            right = i * L + (j + 1) % L
            bond_list.append((node, right)) # 해당 노드와 우측 노드 사이 bond

            down = ((i + 1) % L) * L + j
            bond_list.append((node, down)) # 해당 노드와 하측 노드 사이 bond
    return bond_list

def bond_percolation(L):
    N = L * L
    ptr = [-1] * N  # root 초기화
    size = [1] * N  # size 초기화
    G_list = []
    P_list = []

    bond_list = bonds(L)
    random.shuffle(bond_list)  # bond 순서 무작위로

    for i in range(len(bond_list)):  # 무작위 순서의 bond를 읽고
        P_list.append((i+1)/(2*N))
        node1, node2 = bond_list[i]  # 해당 bond로 연결된 두 노드를 확인
        union(ptr, size, node1, node2)  # union으로 병합

        # 현재 단계의 giant cluster size ratio를 계산
        G_list.append(max(size) / N)

    # 윗줄과 아랫줄이 연결되어 있는지 확인(퍼콜레이션 확인)
    top_row = {find(ptr, i) for i in range(L)}
    bottom_row = {find(ptr, N - 1 - i) for i in range(L)}
    if top_row & bottom_row:  # 두 줄 사이 교집합이 있다면 성공
        return P_list, G_list  # giant cluster size ratio

    return P_list, G_list  # 만약 퍼콜레이션이 발생하지 않더라도 전체 리스트 반환

In [64]:
'''
P5, G5 = bond_percolation(5)
P50, G50 = bond_percolation(50)
P150, G150 = bond_percolation(150)
# G256 = bond_percolation(256)

plt.plot(P5, G5, label='L=5')
plt.plot(P50, G50, label='L=50')
plt.plot(P150, G150, label='L=150')
# plt.plot(G256, label='L=256')

plt.xlabel('Bond Steps')
plt.ylabel('Giant Cluster Size Ratio')
plt.title('Giant Cluster Size Ratio vs Bond Steps for Different L')

plt.legend()
plt.show()
'''

"\nP5, G5 = bond_percolation(5)\nP50, G50 = bond_percolation(50)\nP150, G150 = bond_percolation(150)\n# G256 = bond_percolation(256)\n\nplt.plot(P5, G5, label='L=5')\nplt.plot(P50, G50, label='L=50')\nplt.plot(P150, G150, label='L=150')\n# plt.plot(G256, label='L=256')\n\nplt.xlabel('Bond Steps')\nplt.ylabel('Giant Cluster Size Ratio')\nplt.title('Giant Cluster Size Ratio vs Bond Steps for Different L')\n\nplt.legend()\nplt.show()\n"

## 0613

### Swendsen-Wang algorithm (Cluster flip)

In [65]:
# 주기적 경계 조건을 처리하기 위한 이웃 함수
def get_neighbors(i, j, L):
    return [
        ((i-1) % L, j),  # 위쪽
        ((i+1) % L, j),  # 아래쪽
        (i, (j-1) % L),  # 왼쪽
        (i, (j+1) % L)   # 오른쪽
    ]

# 루트를 찾는 함수
def find(ptr, x):
    if ptr[x] < 0:
        return x
    else:
        ptr[x] = find(ptr, ptr[x]) # 경로 압축
        return ptr[x]

# 두 집합을 합치는 함수
def union(ptr, r1, r2, big):
    if r2 != r1:
        if ptr[r1] > ptr[r2]:
            ptr[r2] += ptr[r1]
            ptr[r1] = r2
            r1 = r2
        else:
            ptr[r1] += ptr[r2]
            ptr[r2] = r1

        if -ptr[r1] > big:
            big = -ptr[r1]
    return r1, r2, big

# 4N/2 = 2N 개의 bond이므로 right, down 두 가지 방향으로 찾아본다
def bonds(L):
    bond_list = []
    for i in range(L):
        for j in range(L):
            node = i * L + j

            right = i * L + (j + 1) % L
            bond_list.append((node, right)) # 해당 노드와 우측 노드 사이 bond

            down = ((i + 1) % L) * L + j
            bond_list.append((node, down)) # 해당 노드와 하측 노드 사이 bond
    return bond_list

def cluster_flip(array, beta):
    p = 1 - np.exp(-2 * beta)  # p=1-exp(-2*beta)
    L = int(np.sqrt(len(array)))  # assuming array is 1D
    N = L * L
    ptr = [-1] * N  # root 초기화
    size = [1] * N  # size 초기화
    bond_list = bonds(L)  # 본드 생성

    big = 0

    # 클러스터 생성
    for bond in bond_list:
        node, neighbor = bond
        if array[node] == array[neighbor]:
            # 같은 스핀이라면 p의 확률로 accept, 1-p의 확률로 reject
            if np.random.rand() < p:
                r1 = find(ptr, node)
                r2 = find(ptr, neighbor)
                r1, r2, big = union(ptr, r1, r2, big)

    clusters = set()  # 클러스터 사전

    for i in range(N):
        r1 = find(ptr, i)
        clusters.add(r1)

    # 각각의 클러스터를 flip
    for cluster in clusters:
        if random.random() < 0.5:
            for i in range(N):
                if find(ptr, i) == cluster:
                    array[i] *= -1

    return array

In [66]:
'''
betas = np.arange(0.0, 1.1, 0.1).tolist() # x축 plot을 위해 beta 리스트 생성

# 5x5
avg_Ss5 = []  # |<S>|

for beta in betas:
    Ss = []
    array = np.random.choice([-1, 1], size=(5*5))
    for _ in range(10000):
        array = cluster_flip(array, beta)
        s = np.mean(array)
        Ss.append(abs(s))
    avg_Ss5.append(np.mean(Ss))

plt.plot(betas, avg_Ss5, label='L=5')
plt.xlabel('Beta')
plt.ylabel('m')
plt.legend()
plt.show()
'''

"\nbetas = np.arange(0.0, 1.1, 0.1).tolist() # x축 plot을 위해 beta 리스트 생성\n\n# 5x5\navg_Ss5 = []  # |<S>|\n\nfor beta in betas:\n    Ss = []\n    array = np.random.choice([-1, 1], size=(5*5))\n    for _ in range(10000):\n        array = cluster_flip(array, beta)\n        s = np.mean(array)\n        Ss.append(abs(s))\n    avg_Ss5.append(np.mean(Ss))\n\nplt.plot(betas, avg_Ss5, label='L=5')\nplt.xlabel('Beta')\nplt.ylabel('m')\nplt.legend()\nplt.show()\n"